# Settings

In [4]:
import os
from dotenv import load_dotenv

load_dotenv()  # .env 파일에서 환경 변수를 불러옵니다.

# api_key = os.getenv("OPENAI_API_KEY")
# print(f"API Key: {api_key}")
google_api_key = os.getenv("GOOGLE_API_KEY")
search_engine_key = os.getenv("GOOGLE_SEARCH_ENGINE")

In [ ]:
#나무크롤러 업데이트시에만 사용
# import requests

# urls = {
#     "namu_crawler.py":"https://raw.githubusercontent.com/lymanstudio/proj_artist_info_gen/main/namu_crawler.py",
#     "namu_loader.py":"https://raw.githubusercontent.com/lymanstudio/proj_artist_info_gen/main/namu_loader.py"
# }

# for key, val in urls.items():    
#     r = requests.get(val)

#     # 다운로드한 파일을 현재 디렉토리에 저장
#     with open(key, "w", encoding='utf-8') as file:
#         file.write(r.text)


# Scraping

## bigquery에서 사전에 저장한 URL 데이터 가져오기

In [2]:
!gcloud auth application-default login
# %pip install --upgrade google-cloud-bigquery pandas db-dtypes

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login&state=grRD1ivlf8RC640NsGj1o18Ji6u2tZ&access_type=offline&code_challenge=L_MN7032xYNzBqhnTdE3wuoUPVc8sz9gGBVMBAlvu3k&code_challenge_method=S256


Credentials saved to file: [C:\Users\tkdgu\AppData\Roaming\gcloud\application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).

Quota project "wev-dev-analytics" was added to ADC which can be used by Google client libraries for billing and quota. Note that some services may still bill the project owning the resource.


In [5]:
from google.cloud import bigquery
import pandas as pd

# 내가 작업하고자 하는 GCP 프로젝트, region, dataset, table id 설정
PROJECT_ID = "wev-dev-analytics"
REGION = "asia-northeast3"
DATASET_ID = "namu_wiki"
TABLE_ID = "art_info_url"

# Initialize a BigQuery client
client = bigquery.Client(project=PROJECT_ID)

# Define the dataset and table you want to import
table_ref = client.dataset(DATASET_ID).table(TABLE_ID)

# Query the table and convert it to a DataFrame
def import_table_to_dataframe(client, table_ref):
    # Construct a BigQuery client object.
    table = client.get_table(table_ref)  # API call

    print(f"Downloading {table.num_rows} rows from {table_ref.path}")

    # Load the table into a Pandas DataFrame
    query = f'SELECT * FROM `{PROJECT_ID}.{DATASET_ID}.{TABLE_ID}`'
    dataframe = client.query(query).to_dataframe()

    return dataframe

# Import the table to a DataFrame
urls = import_table_to_dataframe(client, table_ref)

# Print the DataFrame
print(urls.head())

  we_art_id we_art_name                      url
0         7          CL   https://namu.wiki/w/CL
1        45          XG   https://namu.wiki/w/XG
2        77         3YE  https://namu.wiki/w/3YE
3       129         EXO  https://namu.wiki/w/EXO
4      1648         NOA  https://namu.wiki/w/NOA


c:\Users\tkdgu\OneDrive\문서\Github\idol_info_finder\venv\lib\site-packages\google\cloud\bigquery\table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


## URL에 저장된 데이터 크롤링 후 빅쿼리 테이블에 적재

In [2]:
#코드 흐름
#나무로더 인스턴스 생성
#위키페이지 크롤링
#테이블에 저장
#루프로 진행

In [22]:
import os
import json
import time
from namu_loader import NamuLoader
import textwrap

from google.cloud import bigquery

import openai
from langchain_community.embeddings import OpenAIEmbeddings
# from langchain_google_vertexai import VertexAIEmbeddings
# from langchain_google_community import BigQueryVectorStore

# BigQuery
def load_data_to_bigquery(client, json_data, project_id, dataset_id, table_id, region, write_disposition, artist_info, page_url):
    
    # print(json_data)
    
    # metadata 는 한글이 섞여있으므로 ensure_ascii 옵션을 False 로 설정한다.
    # artist_info, page_url 은 크롤링된 정보에서 가져오는 것이 아니므로 수동으로 넣어준다.
    for item in json_data:
        item['abs_page_toc_item'] = item['metadata']['abs_page_toc_item']
        item['toc_item'] = item['metadata']['toc_item']
        item['metadata'] = json.dumps(item['metadata'], ensure_ascii=False)
        item['artist_info'] = artist_info
        item['page_url'] = page_url
    
    table_ref = client.dataset(dataset_id, project=project_id).table(table_id)
    job_config = bigquery.LoadJobConfig()
    job_config.write_disposition = write_disposition
    
    load_job = client.load_table_from_json(
        json_data, table_ref, location=region, job_config=job_config
    )
    
    load_job.result()  
    print(f'Loaded {len(json_data)} rows into {project_id}:{dataset_id}.{table_id}')

def push_art_info_to_bigquery(art_info, max_hop = 1):
    
	  # NamuLoader 를 사용해서 url 정보를 크롤링한다.
    # url = 'https://namu.wiki/w/(%EC%97%AC%EC%9E%90)%EC%95%84%EC%9D%B4%EB%93%A4?from=%EC%97%AC%EC%9E%90%EC%95%84%EC%9D%B4%EB%93%A4'
    # max_hop = 1
    print(art_info['we_art_name'],' : ',art_info['url'])
    url = art_info['url']
    verbose = True
    try:
        loader = NamuLoader(url=url, max_hop=max_hop, verbose=verbose)
    except:
        return
    # print(url)

		# 크롤링한 데이터를 documents 에 append 
    documents = []
    for doc in loader.lazy_load():
        documents.append({
            "page_content": doc.page_content,
            "metadata": doc.metadata
        })
    
    # 내가 작업하고자 하는 GCP 프로젝트, region, dataset, table id 설정
    PROJECT_ID = "wev-dev-analytics"
    REGION = "asia-northeast3"
    DATASET_ID = "namu_wiki"
    TABLE_ID = "namu_string_result_02"

    # 빅쿼리에 저장할 테이블의 schema 정의
    client = bigquery.Client(project=PROJECT_ID)
    schema = [
      bigquery.SchemaField("page_url", "STRING"),
      bigquery.SchemaField("artist_info", "STRING"),
      bigquery.SchemaField("metadata", "STRING"),
      bigquery.SchemaField("page_content", "STRING"),
      bigquery.SchemaField("toc_item", "STRING"),
      bigquery.SchemaField("abs_page_toc_item", "STRING"),
      ]
    
    dataset_ref = client.dataset(DATASET_ID)
    dataset = bigquery.Dataset(dataset_ref)
    dataset.location = REGION

    # 데이터셋 생성 (이미 존재하는 경우 생략)
    try:
        client.create_dataset(dataset)
        print(f"Created dataset {DATASET_ID} in {REGION}")
    except:
        print(f"Dataset {DATASET_ID} already exists in {REGION}")
    
    # 테이블 생성 (이미 존재하는 경우 생략)
    table_ref = dataset_ref.table(TABLE_ID)
    table = bigquery.Table(table_ref, schema=schema)

    try:
        client.create_table(table)
        print(f"Created table {TABLE_ID} in dataset {DATASET_ID}")
    except:
        print(f"Table {TABLE_ID} already exists in dataset {DATASET_ID}")

		# 넣고 싶은 ARTIST_INFO, PAGE_URL 값을 기입해준다.
    ARTIST_INFO = art_info['we_art_name']
    PAGE_URL = url

		# 각 파라미터를 기입해준다. WRITE_APPEND 은 테이블에 데이터가 append 되고, WRITE_TRUNCATE 을 기입하면 overwrite 된다.
    load_data_to_bigquery(client, documents, PROJECT_ID, DATASET_ID, TABLE_ID, REGION, bigquery.WriteDisposition.WRITE_APPEND, ARTIST_INFO, PAGE_URL) # WRITE_APPEND, WRITE_TRUNCATE

In [ ]:
push_art_info_to_bigquery()

In [11]:
for idx, url in urls.iterrows():
    # print(url['url'])
    push_art_info_to_bigquery(url)

WAYV  :  https://namu.wiki/w/WayV
/w/WayV

Document of /w/WayV


================== Table of Contents  ==================

0. PROFILE
1. 개요
2. 상세
3. 멤버
	3.1. 전 멤버
4. 음반
	4.1. 로고
	4.2. 뮤직비디오
5. 특징
	5.1. 실력
		5.1.1. 보컬
		5.1.2. 랩
		5.1.3. 댄스
6. 유닛
	6.1. WayV-KUN&XIAOJUN
	6.2. WayV-TEN&YANGYANG
7. 공연 및 행사
	7.1. 합동
8. 활동
	8.1. 2019년
	8.2. 2020년
	8.3. 2021년
	8.4. 2022년
	8.5. 2023년
	8.6. 2024년
9. 출연 목록
	9.1. 방송
	9.2. 단독 리얼리티
	9.3. V LIVE
	9.4. 콘텐츠
10. 잡지 화보
11. 광고
12. 홍보대사
13. 수상 경력
	13.1. 시상식
	13.2. 음악 방송 1위
14. 논란
15. 출신지 및 언어 관련
16. 여담
17. 역대 프로필 사진
EOD. FOOTNOTES
/w/WayV
Main Doc /w/WayV                                            Start (hop : 0/1)
/w/WayV/%EC%9D%8C%EB%B0%98
  Sub Doc /w/WayV/%EC%9D%8C%EB%B0%98               Start (hop : 1/1)
  Sub Doc /w/WayV/%EC%9D%8C%EB%B0%98               Done! (hop : 1/1, elapsed_time: 1.0 seconds)
/w/WayV/%EB%AE%A4%EC%A7%81%EB%B9%84%EB%94%94%EC%98%A4
  Sub Doc /w/WayV/%EB%AE%A4%EC%A7%81%EB%B9%84%EB%94%94%EC%98%A4 Start (hop : 1/1)
  Sub Doc /w/WayV/